# VGG Face - 2

# Check GPU version.

In [0]:
!nvidia-smi

# Mount google drive.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Install TensorFlow-1.14 GPU.

In [0]:
# Select TensorFlow-1.x version.
%tensorflow_version 1.x

# Uninstall previous TensorFlow version.
!pip uninstall tensorflow -y 
!pip uninstall tensorflow-gpu -y

# Install TensorFlow-1.14 and Keras-2.2.4.
!pip install --upgrade tensorflow-gpu==1.14.0
!pip install --upgrade tensorflow==1.14.0
!pip install --upgrade keras==2.2.4
!pip install --upgrade scipy

# Restart the runtime.

# Set the root directory.

In [0]:
import os

root_dir = '/content/'
os.chdir(root_dir)

!ls -al

# Import TensorFlow-1.14.

In [0]:
try:
  %tensorflow_version 1.x
except Exception:
  pass

import numpy as np
np.random.seed(7)

import tensorflow as tf
print(tf.__version__)

In [0]:
!pip install keras_vggface

In [0]:
image_shape = (224, 224, 3)
#model_filename = '/content/drive/My Drive/models/resnet50/vgg_face2.h5'
aligned_image_dir = '/content/drive/My Drive/aligned_images/'
test_image_dir = '/content/drive/My Drive/test_images/'

In [0]:
from keras_vggface.vggface import VGGFace

resnet50_features = VGGFace(model='resnet50', include_top=False, input_shape=image_shape, pooling='avg')

In [0]:
from keras_vggface import utils

def normalize_image(image_filename): 
  input_image = image.load_img(image_filename, target_size=(image_shape[0], image_shape[1])) 
  output_image = image.img_to_array(input_image)
  output_image = np.expand_dims(output_image, axis=0) 
  output_image = utils.preprocess_input(output_image, version=1)
  return( output_image )

In [0]:
def compute_image_features(image_filename): 
  current_image = normalize_image(image_filename)
  current_features = resnet50_features.predict(current_image)
  current_features = current_features[0]  
  return( current_features )

In [0]:
from keras.preprocessing import image

def compute_features(aligned_image_dir):
  image_features = {}
  image_filenames = os.listdir(aligned_image_dir)
  for image_filename in image_filenames:
    identifier = image_filename.split('.jpg')
    identifier = identifier[0]

    image_path = os.path.join(aligned_image_dir, image_filename)
    current_features = compute_image_features(image_path)

    image_features[identifier] = current_features
  return(image_features)

In [0]:
image_features = compute_features(aligned_image_dir)

In [0]:
import scipy.spatial.distance as distance

def identify_person(image_features, current_features, threshold=100):
  person_name = 'unknown'
  minimum_distance = float('inf')
  for person in image_features:
    #print(person)
    person_features = image_features[person]
    current_distance = distance.euclidean(person_features, current_features)
    if(current_distance < minimum_distance):
      minimum_distance = current_distance
      person_name = person

  if(minimum_distance > threshold):
    person_name = 'unknown'

  return(person_name, minimum_distance)

In [0]:
def identify_persons(image_features, test_image_dir):
  image_filenames = os.listdir(test_image_dir)
  for image_filename in image_filenames:
    #print(image_filename)
    identifier = image_filename.split('.jpg')
    identifier = identifier[0]
    #print(identifier)

    image_path = os.path.join(test_image_dir, image_filename)
    current_features = compute_image_features(image_path)
    person_name, minimum_distance = identify_person(image_features, current_features)
    print('**************************************************')
    print('ground truth -', identifier)
    print('predicted -',person_name)
    print('distance -',minimum_distance)
    print('**************************************************')

In [0]:
identify_persons(image_features, test_image_dir)